# Gemma3-270M Fine-Tuning Using SFT

## Imports

In [1]:
!pip install transformers datasets torch peft
!pip install -q kagglehub
!pip install wandb

In [2]:
import wandb, os

import huggingface_hub
huggingface_hub.notebook_login()

try:
  from kaggle_secrets import UserSecretsClient
  user_secrets = UserSecretsClient()
  os.environ['KAGGLE_KEY'] = user_secrets.get_secret("KAGGLE_KEY")
  os.environ['KAGGLE_USERNAME'] = user_secrets.get_secret("KAGGLE_USERNAME")
  os.environ['WANDB_API_KEY'] = user_secrets.get_secret("WANDB_API_KEY")
except:
  try:
    from google.colab import userdata
    os.environ['KAGGLE_KEY'] = userdata.get("KAGGLE_KEY")
    os.environ['KAGGLE_USERNAME'] = userdata.get("KAGGLE_USERNAME")
    os.environ['WANDB_API_KEY'] = userdata.get("WANDB_API_KEY")
  except:
    print("Could't get secrets")

In [4]:
import os
import shutil
import torch

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the path to the Hugging Face cache directory
hf_cache_home = os.path.expanduser("~/.cache/huggingface/hub")

# Check if the cache directory exists and remove it
if os.path.exists(hf_cache_home):
    print(f"Clearing Hugging Face cache at: {hf_cache_home}")
    shutil.rmtree(hf_cache_home)
    print("Hugging Face cache cleared.")
else:
    print("Hugging Face cache directory not found, nothing to clear.")

!pip install transformers --upgrade

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import huggingface_hub

# Log in to Hugging Face (if not already logged in, this will prompt)
huggingface_hub.notebook_login()

model_id = "google/gemma-3-270m-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="cuda",
    trust_remote_code=True,
    attn_implementation="eager" # Explicitly use eager attention to avoid XLA issues
)

model.to(device)

print("Model and tokenizer loaded successfully.")

Clearing Hugging Face cache at: /root/.cache/huggingface/hub
Hugging Face cache cleared.


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/536M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Model and tokenizer loaded successfully.


In [5]:
from matplotlib import pyplot as plt

# Helper function to visualize performance during training
def plot_training_curves(train_losses, val_accuracies):
    """Plot training loss and validation accuracy curves.

    Parameters
    ----------
    train_losses : list of float
        Training loss values for each epoch. Should have one value per epoch.
    val_accuracies : list of float
        Validation accuracy values for each epoch. Should have same length as
        train_losses. Accuracy values should be between 0 and 1 (or 0 and 100
        if using percentages).

    Returns
    -------
    None
        Displays matplotlib figure with two subplots showing training curves.

    Examples
    --------
    >>> train_losses = [0.8, 0.6, 0.4, 0.3, 0.2]
    >>> val_accuracies = [0.75, 0.80, 0.85, 0.87, 0.88]
    >>> plot_training_curves(train_losses, val_accuracies)
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    ax1.plot(train_losses)
    ax1.set_title('Training Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.grid(True)

    ax2.plot(val_accuracies)
    ax2.set_title('Validation Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

## MATH dataset

In [6]:
from datasets import load_dataset

ds = load_dataset("qwedsacf/competition_math")
ds_split = ds['train'].train_test_split(test_size=0.10, seed=2025)

train_val_split = ds_split['train'].train_test_split(test_size=0.1111, seed=2025)

math_train = train_val_split['train']
math_val   = train_val_split['test']
math_test  = ds_split['test']

print("Train size:", len(math_train))
print("Val size:",   len(math_val))
print("Test size:",  len(math_test))

README.md: 0.00B [00:00, ?B/s]

Train size: 10000
Val size: 1250
Test size: 1250


In [7]:
#Sample data
print(math_train[1])
print(math_train[2])
print(math_train[3])

{'problem': 'For how many integer values of $a$ does the equation $x^2 + ax + 5a = 0$ have integer solutions for $x$?', 'level': 'Level 5', 'type': 'Algebra', 'solution': "Suppose the roots of the quadratic are given by $m$ and $n$. Note that $$(x-m)(x-n) = x^2 - (m+n)x + mn = x^2 + ax + 5a,$$ and setting coefficients equal, it follows that  \\begin{align*}\nm + n &= -a \\\\\nmn &= 5a\n\\end{align*} (This also follows directly from Vieta's formulas.) Notice that the $a$ can be canceled by either dividing or noting that $$0 = 5a + 5 \\cdot (-a) = mn + 5(m+n).$$\n\nSimon's Favorite Factoring Trick can now be applied: $$mn + 5m + 5n + 25 = (m+5)(n+5) = 25.$$ It follows that $m+5$ and $n+5$ are divisors of $25$, whose pairs of divisors are given by $\\pm \\{(1,25),(5,5),(25,1)\\}$. Solving, we see that $(m,n)$ is in the set $$\\{(-4,20),(0,0),(20,-4),(-6,-30),(-10,-10),(-30,-6)\\}.$$ However, the two pairs of symmetric solutions yield redundant values for $a$, so it follows that the answer

The inspection of `math_train` entries confirmed the presence of numerous LaTeX commands and special characters such as `$`, `\`, `\begin{align*}` (and `\end{align*}`), `\\`, `\cdot`, `\ldots`, `\frac`, `\ge`, `\ge`, `\boxed`, `\binom`, `\times`, `\%`, and others. These need to be removed or replaced to make the text more readable and suitable for natural language processing tasks.

Now, let's create a cleaning function to handle these patterns. We'll start with a basic function and refine it as needed.

TODO: Cleaning MATH dataset

In [8]:
import json
import re
from typing import Dict, Any, Optional

# ============================================
# STEP 1: BASIC CLEANING
# ============================================

def clean_text(text: str) -> str:
    """Basic whitespace normalization."""
    if not text:
        return ""

    # Normalize whitespace
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r'\n\s*\n\s*\n+', '\n\n', text)

    # Clean up lines
    lines = text.split('\n')
    lines = [line.strip() for line in lines]
    text = '\n'.join(lines)

    return text.strip()

def add_cleaned_fields(data):
    """Add cleaned_problem and cleaned_solution fields to dataset."""
    for record in data:
        record['cleaned_problem'] = clean_text(record.get('problem', ''))
        record['cleaned_solution'] = clean_text(record.get('solution', ''))
    return data


# ============================================
# STEP 2: ANSWER EXTRACTION
# ============================================

def extract_answer(text: str) -> Optional[str]:
    """
    Extract final answer from solution text.
    Priority order:
        1. \\boxed{...} - standard competition math format
        2. Final Answer: ... - explicit format
        3. Last number in text - fallback only
    """
    if not text:
        return None

    # 1. Extract from \boxed{...} - HIGHEST PRIORITY
    # Use a more sophisticated approach to handle nested braces
    boxed_content = extract_boxed_content(text)
    if boxed_content:
        answer = clean_answer_text(boxed_content)
        if answer:
            return answer

    # 2. Extract from "Final Answer:" pattern
    final_answer_pattern = r'[Ff]inal [Aa]nswer[:\s]+(.+?)(?:\n|$)'
    final_matches = re.findall(final_answer_pattern, text)
    if final_matches:
        answer = clean_answer_text(final_matches[-1])
        if answer:
            return answer

    # 3. Look for answer after common phrases
    answer_phrases = [
        r'[Tt]he answer is[:\s]+(.+?)(?:\.|$)',
        r'[Tt]herefore[,\s]+(.+?)(?:\.|$)',
        r'[Ss]o[,\s]+the (?:number|answer|value)[^:]*:[:\s]+(.+?)(?:\.|$)',
    ]

    for pattern in answer_phrases:
        matches = re.findall(pattern, text)
        if matches:
            answer = clean_answer_text(matches[-1])
            if answer:
                return answer

    # 4. LAST RESORT: Find last number-like pattern
    # Split by sentences/lines to get context
    sentences = re.split(r'[.\n]', text)
    for sentence in reversed(sentences):
        # Look for standalone numbers or simple expressions
        number_patterns = [
            r'\$([^$]+)\$',  # Math mode
            r'(-?\d+(?:\.\d+)?(?:/\d+)?)',  # Simple numbers/fractions
        ]

        for pattern in number_patterns:
            matches = re.findall(pattern, sentence)
            if matches:
                answer = clean_answer_text(matches[-1])
                if answer:
                    return answer

    return None


def extract_boxed_content(text: str) -> Optional[str]:
    """
    Extract content from \\boxed{...} handling nested braces correctly.
    Returns the content of the LAST \\boxed{} in the text.
    """
    # Find all \boxed{ positions
    pattern = r'\\boxed\{'
    matches = list(re.finditer(pattern, text))

    if not matches:
        return None

    # Process from the last match backwards
    for match in reversed(matches):
        start = match.end()  # Start after the opening brace
        brace_count = 1
        i = start

        # Find matching closing brace
        while i < len(text) and brace_count > 0:
            char = text[i]

            # Skip escaped braces
            if i > 0 and text[i-1] == '\\':
                i += 1
                continue

            if char == '{':
                brace_count += 1
            elif char == '}':
                brace_count -= 1
            i += 1

        if brace_count == 0:
            # Successfully found matching brace
            content = text[start:i-1]
            return content.strip()

    return None


def clean_answer_text(text: str) -> str:
    """Clean extracted answer text - preserve LaTeX commands."""
    if not text:
        return ""

    text = text.strip()

    # Remove outer dollar signs only
    text = re.sub(r'^\$+', '', text)
    text = re.sub(r'\$+$', '', text)

    # Clean whitespace but preserve structure
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove trailing punctuation (but not mathematical operators)
    text = re.sub(r'[.,;!?]+$', '', text)

    # Remove any closing $ that might be left
    text = text.rstrip('$')

    return text


def extract_numeric_answer(text: str) -> Optional[str]:
    """
    Extract purely numeric answer (integers, decimals, fractions).
    Use this for problems expecting numeric answers only.
    """
    answer = extract_answer(text)
    if not answer:
        return None

    # Try to extract numeric value from answer
    # Handle fractions
    frac_match = re.search(r'\\frac\{(-?\d+)\}\{(-?\d+)\}', answer)
    if frac_match:
        num, den = frac_match.groups()
        return f"{num}/{den}"

    # Handle simple fractions
    simple_frac = re.search(r'(-?\d+)/(-?\d+)', answer)
    if simple_frac:
        return simple_frac.group(0)

    # Handle exponents
    exp_match = re.search(r'(-?\d+)\^?\{?(-?\d+)\}?', answer)
    if exp_match and '^' in answer:
        base, exp = exp_match.groups()
        return f"{base}^{exp}"

    # Handle decimals and integers
    num_match = re.search(r'-?\d+(?:\.\d+)?', answer)
    if num_match:
        return num_match.group(0)

    # Return original cleaned answer if no numeric pattern found
    return answer

# ============================================
# STEP 2b: EXTRACT REASONING
# ============================================

def extract_reasoning(text: str) -> str:
    """Extract everything before the final answer."""
    if not text:
        return ""

    # Split at Final Answer
    match = re.split(r'[Ff]inal [Aa]nswer[:\s]+', text)
    if len(match) > 1:
        return match[0].strip()

    # Try splitting before a boxed answer
    boxed_match = re.split(r'\\boxed\{', text)
    if len(boxed_match) > 1:
        return boxed_match[0].strip()

    # Fallback: return all but last 1–2 lines
    lines = text.strip().split("\n")
    if len(lines) > 2:
        return "\n".join(lines[:-2]).strip()

    return text.strip()


# ============================================
# STEP 3: ADD FINAL ANSWER TO SOLUTIONS
# ============================================

def reformat_solution(record: Dict[str, Any]) -> Dict[str, Any]:
    """Add 'Final Answer: X' to cleaned_solution."""
    solution = record.get('cleaned_solution', '')

    # Extract answer
    final = extract_answer(solution)

    if final is None:
        # If no answer found, try to extract from original solution
        final = extract_answer(record.get('solution', ''))

    if final is None:
        # Fallback: mark as unable to extract
        final = "[UNABLE_TO_EXTRACT]"

    # Check if "Final Answer:" already exists
    if not re.search(r'[Ff]inal [Aa]nswer:', solution):
        record['cleaned_solution'] = solution + f"\n\nFinal Answer: {final}"

    record["final_answer"] = extract_answer(solution) or "[UNABLE]"
    record["cleaned_reasoning"] = extract_reasoning(solution)

    return record


# ============================================
# MAIN PROCESSING PIPELINE
# ============================================

def process_example(example):
    # 1. Clean problem+solution
    cleaned_problem = clean_text(example.get("problem", ""))
    cleaned_solution = clean_text(example.get("solution", ""))

    # 2. Add cleaned fields
    example["cleaned_problem"] = cleaned_problem
    example["cleaned_solution"] = cleaned_solution

    # 3. Extract answer and attach "Final Answer:"
    example = reformat_solution(example)

    return example

# ============================================
# TESTING UTILITIES
# ============================================

def test_extraction(test_cases = [
        ("The answer is $\\boxed{42}$", "42"),
        ("Therefore, we get $\\boxed{2^{2008} - 2}$", "2^{2008} - 2"),
        ("Final Answer: 17", "17"),
        ("So the value is \\frac{3}{4}.", "3/4"),
        ("The result is -15.", "-15"),
    ]):
    """Test answer extraction on common formats."""

    print("\n" + "="*60)
    print("Testing Answer Extraction:")
    print("="*60)

    for text, expected in test_cases:
        extracted = extract_answer(text)
        status = "✓" if extracted == expected else "✗"
        print(f"{status} Input: {text[:50]}...")
        print(f"  Expected: {expected}")
        print(f"  Got: {extracted}")
        print()

# Uncomment to run tests
#test_extraction([(" The answer is, as you can see, \frac{1}{5} because", fix_latex("\frac{1}{5}")),(math_train[1]["solution"],"4"), (math_train[2]["solution"],"2^{2008} -2"), (math_train[0]["solution"],"\sqrt{10}"), (math_train[3]["solution"], "\frac{5}{8}")])

In [9]:
#applying to training/testing splits
math_train = math_train.map(process_example, desc="Cleaning train split")
math_val   = math_val.map(process_example, desc="Cleaning val split")
math_test  = math_test.map(process_example, desc="Cleaning test split")


print("Cleaning applied to math_train, math_test and math_val datasets.")
print("Sample of math_train after cleaning:")
print(math_train[0]['problem'])
print(math_train[0]['cleaned_problem'])
print(math_train[0]['solution'])
print(math_train[0]['cleaned_solution'])

Cleaning applied to math_train, math_test and math_val datasets.
Sample of math_train after cleaning:
What is the distance between the two intersections of $y=x^2$ and $x+y=1$?
What is the distance between the two intersections of $y=x^2$ and $x+y=1$?
To find the $x$-coordinates of the intersections, substitute $x^2$ for $y$ in $x+y=1$ and solve for $x$, resulting in  \begin{align*}
x+x^2&=1 \\
\Rightarrow \qquad x^2+x-1&=0 \\
\Rightarrow \qquad x&=\frac{-1\pm\sqrt{1+4}}2=\frac{-1\pm\sqrt5}2\\
\end{align*}Using each of these coordinates to solve for $y$ gives us the intersections at $\left(\frac{-1+\sqrt5}2,\frac{3-\sqrt5}2\right)$ and $\left(\frac{-1-\sqrt5}2,\frac{3+\sqrt5}2\right)$.  Using the distance formula, we have \begin{align*}
&\sqrt{ \left(\frac{-1+\sqrt5}{2}-\frac{-1-\sqrt5}{2}\right)^2 + \left(\frac{3-\sqrt5}2-\frac{3+\sqrt5}2\right)^2 }\\
&\qquad=\sqrt{\left(\frac{2\sqrt5}2\right)^2 + \left(-\frac{2\sqrt5}2\right)^2}\\
&\qquad=\sqrt{ 2\sqrt5^2 }\\
&\qquad=\boxed{\sqrt{10}

In [10]:
math_train[0]

{'problem': 'What is the distance between the two intersections of $y=x^2$ and $x+y=1$?',
 'level': 'Level 5',
 'type': 'Algebra',
 'solution': 'To find the $x$-coordinates of the intersections, substitute $x^2$ for $y$ in $x+y=1$ and solve for $x$, resulting in  \\begin{align*}\nx+x^2&=1 \\\\\n\\Rightarrow \\qquad x^2+x-1&=0 \\\\\n\\Rightarrow \\qquad x&=\\frac{-1\\pm\\sqrt{1+4}}2=\\frac{-1\\pm\\sqrt5}2\\\\\n\\end{align*}Using each of these coordinates to solve for $y$ gives us the intersections at $\\left(\\frac{-1+\\sqrt5}2,\\frac{3-\\sqrt5}2\\right)$ and $\\left(\\frac{-1-\\sqrt5}2,\\frac{3+\\sqrt5}2\\right)$.  Using the distance formula, we have \\begin{align*}\n&\\sqrt{ \\left(\\frac{-1+\\sqrt5}{2}-\\frac{-1-\\sqrt5}{2}\\right)^2 + \\left(\\frac{3-\\sqrt5}2-\\frac{3+\\sqrt5}2\\right)^2 }\\\\\n&\\qquad=\\sqrt{\\left(\\frac{2\\sqrt5}2\\right)^2 + \\left(-\\frac{2\\sqrt5}2\\right)^2}\\\\\n&\\qquad=\\sqrt{ 2\\sqrt5^2 }\\\\\n&\\qquad=\\boxed{\\sqrt{10}}.\n\\end{align*}',
 'cleaned_pro

## Training Utility Functions

In [11]:
def format_math(example, tokenizer):
    """Format example for training with proper label masking.

    Args:
        example: Dictionary with 'cleaned_problem' and 'cleaned_solution' keys
        tokenizer: HuggingFace tokenizer

    Returns:
        Dictionary with 'input_ids', 'attention_mask', and 'labels'
        Labels are -100 for question tokens (ignored in loss) and
        actual token IDs for answer tokens.
    """
    prompt_template = """Solve the math problem; show your work in a 'Reasoning:' section (LaTeX is allowed) in strictly 2-4 sentences.

You MUST finish with a single line that starts exactly with:

Final Answer:
<final numeric answer only — no LaTeX, no text>

Question:
{question}

Reasoning:
"""
    question_text = prompt_template.format(question=example['cleaned_problem'])
    answer_text = f"{example['cleaned_solution']}"

    # Tokenize question and answer separately
    question_tokens = tokenizer(question_text, add_special_tokens=True, truncation=True,max_length=1024)
    answer_tokens = tokenizer(answer_text, add_special_tokens=False, truncation=True,max_length=1024)

    # Combine
    input_ids = question_tokens['input_ids'] + answer_tokens['input_ids']
    attention_mask = question_tokens['attention_mask'] + answer_tokens['attention_mask']

    # Create labels: -100 for question (ignored), actual tokens for answer
    labels = [-100] * len(question_tokens['input_ids']) + answer_tokens['input_ids']

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }

# Example:
print(f"First Question Tokenized: {tokenizer(math_train[0]['cleaned_problem'], add_special_tokens=True)}")
print()
print(f"First Answer Tokenized: {tokenizer(math_train[0]['cleaned_solution'], add_special_tokens=True)}")
print()
example_formatted = format_math(math_train[0], tokenizer)
print(f"Combined tokenized input: {example_formatted['input_ids']}")
print()
print(f"Prediction targets/labels: {example_formatted['labels']}")

First Question Tokenized: {'input_ids': [2, 3689, 563, 506, 5149, 1534, 506, 1156, 69811, 529, 609, 236762, 236784, 236781, 236884, 236778, 236795, 532, 609, 236781, 236862, 236762, 236784, 236770, 223680], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

First Answer Tokenized: {'input_ids': [2, 2021, 1586, 506, 609, 236781, 4624, 56220, 529, 506, 69811, 236764, 22784, 609, 236781, 236884, 236778, 236795, 573, 609, 236762, 236795, 528, 609, 236781, 236862, 236762, 236784, 236770, 236795, 532, 8974, 573, 609, 236781, 2500, 9113, 528, 621, 1906, 236782, 7341, 9774, 107, 236781, 236862, 236781, 236884, 236778, 36412, 236770, 1704, 107, 236785, 13676, 621, 38453, 1123, 236884, 236778, 236862, 236781, 236772, 236770, 36412, 236771, 1704, 107, 236785, 13676, 621, 38453, 1123, 70812, 1420, 17960, 236770, 236785, 3840, 236785, 4784, 236782, 236770, 236862, 236812, 1807, 236778, 1631, 1420, 17960, 236770, 236785, 3840, 236785, 4784, 236810, 236783

In [15]:
import tqdm
import torch

def eval_math(
    model, tokenizer, data, batch_size=16, max_new_tokens=128,
    verbose=False, num_eval=None
):
    """Evaluate format accuracy, final answer accuracy, and reasoning accuracy."""

    model.eval()

    prompt_template = ("""Solve the math problem; show your work in a 'Reasoning:' section (LaTeX is allowed) in strictly 2-4 sentences.

You MUST finish with a single line that starts exactly with:

Final Answer:
<final numeric answer only — no LaTeX, no text>

Question:
{question}

Reasoning:"""
    )

    if num_eval is None:
        num_eval = len(data)

    # Metrics
    formatted = 0
    correct = 0
    reasoning_correct = 0
    reasoning_comparable = 0  # how many reasoning segments exist to evaluate

    items = list(range(num_eval))

    for start in tqdm.tqdm(range(0, num_eval, batch_size), disable=not verbose):
        batch_idx = items[start:start + batch_size]
        batch = [data[i] for i in batch_idx]

        prompts = [
            prompt_template.format(question=item["cleaned_problem"])
            for item in batch
        ]

        inputs = tokenizer(
            prompts,
            padding=True,
            truncation=True,
            max_length=1024,
            return_tensors="pt"
        ).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
            )

        # Process each output
        for j, item in enumerate(batch):
            input_len = inputs["input_ids"][j].shape[0]
            gen_ids = outputs[j][input_len:]
            gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True)

            # extract answer & reasoning (your existing extract fn)
            pred_ans = extract_answer(gen_text)
            gt_ans  = extract_answer(item["cleaned_solution"])

            pred_reasoning = extract_reasoning(gen_text)
            gt_reasoning   = item.get("cleaned_reasoning", None)

            # ---- FORMAT ACCURACY ----

            # 1. Must contain Reasoning: somewhere
            if "Reasoning:" in gen_text:
                formatted+=1

            # 2. Must contain "Final Answer:" exactly once
            if gen_text.count("Final Answer:") == 1:
                formatted+=1


            # ---- FINAL ANSWER ACCURACY ----
            if pred_ans is not None and gt_ans is not None and pred_ans == gt_ans:
                correct += 1

            # ---- REASONING ACCURACY ----
            if pred_reasoning is not None and gt_reasoning is not None:
                reasoning_comparable += 1
                # strict equality comparison for now
                if pred_reasoning.strip() == gt_reasoning.strip():
                    reasoning_correct += 1

    # ---- METRICS ----
    total = num_eval
    format_rate = formatted / total if total else 0
    answer_accuracy = correct / formatted if formatted else 0
    reasoning_accuracy = reasoning_correct / reasoning_comparable if reasoning_comparable else 0

    print(f"\nFormat rate:        {format_rate:.2%} ({formatted}/{total})")
    print(f"Answer accuracy:    {answer_accuracy:.2%} ({correct}/{formatted})")
    print(f"Reasoning accuracy: {reasoning_accuracy:.2%} ({reasoning_correct}/{reasoning_comparable})")

    return format_rate, answer_accuracy, reasoning_accuracy


In [16]:
#testing eval function
eval_math(model, tokenizer, math_train, verbose=True, num_eval=3)

  0%|          | 0/1 [00:00<?, ?it/s]


AcceleratorError: CUDA error: an illegal memory access was encountered
Search for `cudaErrorIllegalAddress' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [17]:
!nvidia-smi


Wed Dec 10 18:24:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   36C    P0             84W /  400W |    1227MiB /  81920MiB |    100%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## Baseline Model

In [14]:
#evaluating performance on full untrained model
print("Baseline results:")
baseline_format_rate, baseline_accuracy = eval_math(model, tokenizer, math_test)

print(f"Format rate: {baseline_format_rate}")
print(f"Accuracy: {baseline_accuracy}")

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


Baseline results:


RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasGemmStridedBatchedEx(handle, opa, opb, (int)m, (int)n, (int)k, (void*)&falpha, a, CUDA_R_16BF, (int)lda, stridea, b, CUDA_R_16BF, (int)ldb, strideb, (void*)&fbeta, c, std::is_same_v<C_Dtype, float> ? CUDA_R_32F : CUDA_R_16BF, (int)ldc, stridec, (int)num_batches, compute_type, CUBLAS_GEMM_DEFAULT_TENSOR_OP)`

# SFT training

## Train model

In [ ]:
#To keep track of training & validation loss to visualize
class LossTrackerCallback(TrainerCallback):
    def __init__(self, train_losses, val_losses):
        self.train_losses = train_losses
        self.val_losses = val_losses

    def on_epoch_end(self, args, state, control, **kwargs):
        # Logs always contains latest loss
        logs = state.log_history[-1]

        if "loss" in logs:
            self.train_losses.append(logs["loss"])

        if "eval_loss" in logs:
            self.val_losses.append(logs["eval_loss"])

In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, TrainerCallback, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model, TaskType
import torch
from transformers import DataCollatorForLanguageModeling

# Define a simple callback to print the train loss periodically
class PrintLossCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and 'loss' in logs:
            print(f"Step {state.global_step}: Train loss={logs['loss']:.4f}")

def train_math(model, tokenizer, train_data, val_data, test_data,
                batch_size, gradient_accumulation_steps,
                learning_rate, max_epochs, patience,
                lora_r, lora_drop):
    """Fine-tune model with LoRA on MATH."""

    # Set padding token
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id
    model.config.eos_token_id = tokenizer.eos_token_id

    # Apply LoRA
    lora_config = LoraConfig(r=lora_r, lora_alpha=lora_r * 2, #EDIT: acc to gemma notation, lora config
                             target_modules=[
                                 "q_proj", "k_proj", "v_proj", "o_proj",
                                  # "gate_proj", "up_proj", "down_proj" --> unused bec 3x MLPs
                                  ],
                             lora_dropout=lora_drop, bias="none",
                             task_type=TaskType.CAUSAL_LM)
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()


    # --- REQUIRED FOR GEMMA3 + PEFT + GRAD CHECKPOINTING ---
    model.gradient_checkpointing_enable() #must be AFTER LoRA
    model.enable_input_require_grads()
    model.config.use_cache = False #(we use later in eval)
    model.train()

    model.to(device)

    # Format and tokenize with proper masking
    train_tokenized = train_data.map(
        lambda x: format_math(x, tokenizer),
        remove_columns=train_data.column_names
    )
    val_tokenized = val_data.map(
        lambda x: format_math(x, tokenizer),
        remove_columns=val_data.column_names
    )

    # Data collator for padding with label padding (less memory)
    #data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, label_pad_token_id=-100, padding="longest")
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

    train_losses = []
    val_losses = []

    # Training arguments
    args = TrainingArguments(
        output_dir="./math_checkpoints",
        num_train_epochs=max_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=1, #EDIT: VALIDATIOIN IS SMALLER FOR MEMORY ISSUES
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        logging_steps=20,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        fp16=False,
        bf16=True,  #edit: true for TPU usage, smaller GPU usage
        report_to="none",
        gradient_checkpointing=True,
        eval_accumulation_steps=1, #EDIT: again, mem saving for val set
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_tokenized,
        eval_dataset=val_tokenized,
        data_collator=data_collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=patience),
                   PrintLossCallback(),
                   LossTrackerCallback(train_losses, val_losses)],
    )

    # Train
    trainer.train()

    #save training
    trainer.save_model("./sft_model_full")
    tokenizer.save_pretrained("./sft_model_full")

    # Final evaluation
    model.config.use_cache = True
    print("\nFinal test results:")
    eval_math(model, tokenizer, math_test)

    #Final performance visualization
    print("\nPlotting training curves...")
    plot_training_curves(train_losses, val_losses)

    return model

## Setting hyperparameters

In [ ]:
# TODO: Set your hyperparameters here

max_length = 768
batch_size = 1  # Try 2, 4, or 8
gradient_accumulation_steps = 32  # Try 4 or 8
learning_rate = 2e-5 # Try 1e-4, 5e-5, or 1e-5
max_epochs = 1  # Try 5-10
patience = 2  # Try 2 or 3
lora_r = 4  # Try 8, 16, or 32
lora_drop =0.05  # Try 0.05 or 0.1
lora_alpha=16

# Free GPU memory and reload a fresh model for fine-tuning
# This is to ensure that if you run this cell multiple times
# with different hyperparameters, you always start from the
# clean original pretrained model (not a partially fine-tuned one)
import gc
del model
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

#model_id = google/gemma-3-270m-it
#tpu_device = xm.xla_device()
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="cuda"
)
model.to(device)
model.config.use_flash_attn = False

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

model.config.pad_token_id = tokenizer.pad_token_id

print("Fresh model loaded and ready for fine-tuning")

# Run training
model = train_math(
    model=model,
    tokenizer=tokenizer,
    train_data=math_train,
    val_data=math_val, #turn off for now (storage issues)
    test_data=math_test,
    batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    max_epochs=max_epochs,
    patience=patience,
    lora_r=lora_r,
    lora_drop=lora_drop
)

In [ ]:
# Finally, plot training curves
plot_training_curves(train_losses, val_accuracies)

# Ablation Study

diff loRA params

# Deploying with Streamlit

In [ ]:
!pip install streamlit

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model_id = "google/gemma-3-270m-it"  # or your gemma3 model_id

# 1. Load base pretrained model
base = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype="auto",
    device_map="cpu"
)

# 2. Load your adapter weights
model = PeftModel.from_pretrained(base, "./math_checkpoints/checkpoint-313")

# 3. Merge LoRA → full model
model = model.merge_and_unload()

# 4. Save full merged model
model.save_pretrained("final_sft_model")
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.save_pretrained("final_sft_model")

print("Merged model saved to final_sft_model/")


In [ ]:
%%writefile app.py
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_PATH = "final_sft_model"

@st.cache_resource
def load_model():
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_PATH,
        trust_remote_code=True,
        fix_mistral_regex=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )
    return tokenizer, model

tokenizer, model = load_model()


In [ ]:
!streamlit run app.py --server.enableCORS false --server.enableXsrfProtection false
